In [ ]:
from cite_on import *
import numpy as np
from tqdm import tqdm

In [ ]:
data=utils.TSeries('/home/luca/Desktop/501271265_5000.tif',buffer=50,kind='step')
detector=AsyncDetector('wheights/',2,0.8,'0',(512,512))
tracker=Tracker()
extractor=Extractor(workers=46)

In [ ]:
with detector.startup_lock:
    temp_boxes=[]
    with tqdm(total=len(data)) as pbar:
        for _ in range(len(data)):
            frame, proj = data.read()
            if not detector.out_queue.empty():
                boxes=detector.out_queue.get()
                tracker.update(boxes, True)
                temp_boxes=tracker.get_positions()
                pbar.set_postfix(**{'IDs':len(temp_boxes)})
            if proj is not None:
                detector.in_queue.put(proj)
            extractor.extract(frame, temp_boxes)
            pbar.update(1)
        pbar.close()
boxes=tracker.get_positions()
traces=extractor.get_traces()